In [1]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
import gradio as gr
from langchain_groq import ChatGroq
from langchain.chains import ConversationalRetrievalChain



In [2]:
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

pdf_path = '/Users/jsmidt/Documents/Github/GraduateAssistant/resources/092706_1_online.pdf'  # Replace with your PDF path
pdf_text = extract_text_from_pdf(pdf_path)

In [3]:
def split_text_into_chunks(text, chunk_size=2000, chunk_overlap=200):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    return text_splitter.split_text(text)

text_chunks = split_text_into_chunks(pdf_text)

In [4]:

embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

vector_store = Chroma(
    collection_name="disease_info",
    embedding_function=embedding_model,
    persist_directory="./chroma_db"
)

vector_store.add_texts(texts=text_chunks)

/var/folders/kg/3bt8jb_n4vvd559q56xkgl740008fx/T/ipykernel_45181/1267071680.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/var/folders/kg/3bt8jb_n4vvd559q56xkgl740008fx/T/ipykernel_45181/1267071680.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


['163e49e4-2785-4613-bd8b-45b3016ebb20',
 '98bf1f63-3f2b-4f10-9d48-75e394d50cf5',
 'ebe8e398-1a80-4bef-bac2-3c90bd0051e4',
 '9ebbb76e-6e3a-4dcf-925c-d986dde8c2c2',
 '8081e111-145c-47bf-ad3c-cbb7a403946d',
 '171afc11-941b-4f6a-9df3-10bcb4aad692',
 'e84dfbf7-afc1-4381-b732-cefdf6587e1c',
 '9207aea5-8573-41fb-b804-6fe0bcdd2bf6',
 '4f51b7d9-b1f2-46b9-9d97-a9af8a2a87a2',
 '9d6a8ca7-af8a-443d-b3c6-35e7617843a2',
 'e659009e-e1c6-432b-b061-5a89a7de7721',
 'fe769438-1c1d-4b72-a26c-75abff19ae2a',
 'b48e2df5-455a-41e8-a10a-0de1da988b52',
 'f6e983da-95f8-4475-9c08-ac5f7096cc76',
 '38d1822e-6580-45d6-b790-822b6f07edaf',
 '43f4dba2-bea1-428f-b970-e6e7e36598f6',
 '20c6b75a-445c-4f31-b17e-09db0c053997',
 'c6c8c92e-b592-4df4-836a-e88409240f39',
 'e3f3d916-8f80-4ebf-95e9-000937c7cca0',
 '8a8e8129-eb37-479e-b0b8-4a237d6dc69c',
 '0bad8941-2427-4d3c-a0be-561052e50725',
 '4818ddf5-9a46-4fe2-895b-e7bbbb9c9bf2',
 'f70faae0-8a49-4c9c-b660-4d4580af7f5a',
 'faca4c10-fecd-4c42-8685-f49b444cc149',
 '9d6502f7-fe5d-

In [5]:
llm = ChatGroq(model="mixtral-8x7b-32768", temperature=0.1)

In [6]:
retrieval_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vector_store.as_retriever(topk=3),
    return_source_documents=True
)

In [7]:
conversation_history = []

def get_response(user_query):
    response = retrieval_chain({
        "question": user_query,
        "chat_history": conversation_history
    })
    conversation_history.append((user_query, response['answer']))
    return response['answer']

In [8]:
def chat_interface(user_input, history):
    response = get_response(user_input)
    history.append((user_input, response))
    return history, history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    state = gr.State([])
    with gr.Row():
        user_input = gr.Textbox(show_label=False, placeholder="Enter your question...")
        submit_btn = gr.Button("Send")
    submit_btn.click(chat_interface, inputs=[user_input, state], outputs=[chatbot, state])

/Users/jsmidt/Library/Python/3.12/lib/python/site-packages/gradio/components/chatbot.py:282: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


In [9]:
demo.launch()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [10]:
ConversationalRetrievalChain?


Init signature:
ConversationalRetrievalChain(
    *args: Any,
    name: Optional[str] = None,
    memory: Optional[langchain_core.memory.BaseMemory] = None,
    callbacks: Union[list[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None,
    verbose: bool = <factory>,
    tags: Optional[List[str]] = None,
    metadata: Optional[Dict[str, Any]] = None,
    callback_manager: Optional[langchain_core.callbacks.base.BaseCallbackManager] = None,
    combine_docs_chain: langchain.chains.combine_documents.base.BaseCombineDocumentsChain,
    question_generator: langchain.chains.llm.LLMChain,
    output_key: str = 'answer',
    rephrase_question: bool = True,
    return_source_documents: bool = False,
    return_generated_question: bool = False,
    get_chat_history: Optional[Callable[[List[Union[Tuple[str, str], langchain_core.messages.base.BaseMessage]]], str]] = None,
    response_if_no_docs_found: Optional[str] = None,
    ret

/var/folders/kg/3bt8jb_n4vvd559q56xkgl740008fx/T/ipykernel_45181/3916198089.py:4: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = retrieval_chain({
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
